# Ch04 word2vec 속도 개선 

- 어휘가 100만개, 은닉층의 뉴련이 100개인 CBOW모델
- 입력층과 출력층에는 각 100만개의 뉴런이 존재한다. 병목(bottle neck)현상 발생 
    1. 입력층의 원핫 표현과 가중치 행렬 W_in의 곱계산 - 임베딩 계층을 도입하는것으로 해결가능히다. 
    2. 은닉층과 가중치 행렬 W_out의 곱 및 Softmax 계층의 계산 - 네거티브 샘플링이라는 손실함수를 도입하여 해결한다.

## 4.1.1 Embedding 계층
- 자연어 처리에서 단어의 밀집벡터표현을 단어임베딘 혹은 분산표현이라고 한다. 

In [2]:
import numpy as np
W= np.arange(21). reshape(7,3)
print(W)
print(W[2])
print(W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]


In [3]:
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [9]:
class Embedding:
    def __init__(self, W):
        self.params =[W]
        self.grads= [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self,idx):
            dW[word_id] +=dout
        return None
    # idx 배열의 원소 중 값(행번호)이 값은 원속 있다면 dh를 해당 행에 할당시 문제가 발생한다. 
    # 아래는 문제의 코드이다. 
    """"def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        dW[self.idx] = dout
        return None"""

# 4.2 word2vec 개선 2 
- 네거티브 셈플링 > sofrmax 대신 이것을 사용한다면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제가능하다. 
- 은닉층 이후 계산이 가장 오래걸리는 부분은 
    1. 은닉층의 뉴련과 가중치 행렬의 곱
    2. softmax 계층의 계산

In [10]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W) # 임베비드 계층
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## 4.2.5 네거티브 샘플링

In [11]:
print(np.random.choice(10))
print(np.random.choice(10))

1
4


In [17]:
words= ['you', 'say', 'goodbye', 'I',  'hello', '.']
print(np.random.choice(words))
print(np.random.choice(words, size =5 ))
print(np.random.choice(words, size =5 , replace =False)) # 중복없음
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

goodbye
['goodbye' 'goodbye' 'you' 'say' 'goodbye']
['.' 'goodbye' 'say' 'hello' 'I']
I


## 4.2.7 네거티브 샘플링 구현

In [19]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [20]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size) # unigramSampler는 가져옴
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

# 4.3. 개선판 word2vec 학습

In [22]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


## 4.3.2 CBOW 모델 학습 코드 
- 코드 구현 전 학습코드 뒤의 설명을 보니 반나절 걸린다고 써있는걸 봄.. GPU 모드가 있지만, 지금 노트북으로는 어림도없지 ! 
- 주말 내로 MNIST, CBOW 모두 데스크 탑으로 학습을 돌려 놓을예정.. 